Hello 🙌, welcome to my notebook. In this notebook we will try to learn Complete Exploratory Data Analysis (EDA) on Video Game Sales. Also making time series visualization. Feel free if you have any question or suggestion! Thank you!

# Task & Feature Defenition

Feature Defenition

- Rank - Ranking of overall sales
- Name - The games name
- Platform - Platform of the games release (i.e. PC,PS4, etc.)
- Year - Year of the game's release
- Genre - Genre of the game
- Publisher - Publisher of the game
- NA_Sales - Sales in North America (in millions)
- EU_Sales - Sales in Europe (in millions)
- JP_Sales - Sales in Japan (in millions)
- Other_Sales - Sales in the rest of the world (in millions)
- Global_Sales - Total worldwide sales.

Task: Exploraroty Data Analysis (EDA)

# Data Importing & Preview

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

data1 = pd.read_csv('../input/videogamesales/vgsales.csv')

In [ ]:
data1.head()

In [ ]:
data1.info()

In [ ]:
data1.shape

In [ ]:
data1.describe()

In [ ]:
data1.describe(include=['object'])

In [ ]:
'''Missing Value Chart'''
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
data1.isnull().mean(axis=0).plot.barh()
plt.title("Ratio of missing values per columns")

In [ ]:
data1.isnull().values.sum() #total missing values

In [ ]:
data1.isnull().sum().sort_values(ascending=False) #missing values kolom

In [ ]:
data1[data1.isnull().T.any().T].head()

In [ ]:
'''Checking Duplicate'''

print('Dupplicate entries: {}'.format(data1.duplicated().sum()))
# data.drop_duplicates(inplace = True)

In [ ]:
obj = data1.dtypes[data1.dtypes == "object"].index.tolist()
print(obj)

In [ ]:
'''Nunique Columns'''

def nunique_counts(data):
   for i in data.columns:
       count = data[i].nunique()
       print(i, ": ", count)
    
nunique_counts(data1)

In [ ]:
'''Unique Columns'''

def unique_counts(data):
    for i in obj:
        count = data[i].unique()[:5]
        print(i, ": ", count)
    
unique_counts(data1)

In [ ]:
data1.dropna(inplace=True) #removing all missing values

In [ ]:
import numpy as np

data1['Year'] = data1['Year'].astype(np.int64)

In [ ]:
data1['Year'] = pd.to_datetime(data1['Year'].astype(np.int64), format= '%Y' )

In [ ]:
data1['Year'] = data1['Year'].dt.year # getting year

In [ ]:
data1.head()

- Shape of the data is 16598 rows, and 11 features (before dropping missing values)
- Year and Publisher are feature which have missing values. I simply dropped all missing values because later we will make time series which need Year and Publisher
- Then convert Year to Int (it was a float before which have .0 at end of the item). And also convert it again to date time format (year)

# Region Sales Based on Genre

In [ ]:
import plotly.express as px
import plotly.offline as py 
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go 
import plotly.tools as tools
import warnings
from collections import Counter 

custom_aggregation = {}
custom_aggregation["NA_Sales"] = "mean"
custom_aggregation["EU_Sales"] = "mean"
custom_aggregation["JP_Sales"] = "mean"
custom_aggregation["Other_Sales"] = "mean"
custom_aggregation["Global_Sales"] = "mean"

data0 = data1.groupby("Genre").agg(custom_aggregation)
data0.columns = ["North America",'Europe','Japan','Other','Global']
data0['Genre'] = data0.index

na = go.Bar(
    x = data0.Genre.value_counts().index.sort_values(),
    y = data0["North America"],
    name='North America')

eu = go.Bar(
    x = data0.Genre.value_counts().index.sort_values(),
    y = data0["Europe"],
    name='Europe')

jp = go.Bar(
    x = data0.Genre.value_counts().index.sort_values(),
    y = data0["Japan"],
    name='Japan')

oth = go.Bar(
    x = data0.Genre.value_counts().index.sort_values(),
    y = data0["Other"],
    name='Other')

glo = go.Bar(
    x = data0.Genre.value_counts().index.sort_values(),
    y = data0["Global"],
    name='Global')


data = [na, eu, jp, oth, glo]

fig = tools.make_subplots(rows=1, 
                          cols=1)

fig.append_trace(na, 1, 1)
fig.append_trace(eu, 1, 1)
fig.append_trace(jp, 1, 1)
fig.append_trace(oth, 1, 1)
fig.append_trace(glo, 1, 1)


fig['layout'].update(height=500, 
                     width=900, 
                     title='Region Sales Based on Genre', 
                    xaxis_title="Genre",
                    yaxis_title="Mean Sales (in Million)",
                    legend_title="Region")

py.iplot(fig, filename='combined-savings')

- For region sales, we can see that Platfrom Genre was the best in sales, followed by Shooter and Role Playing

# Game Count by Year

In [ ]:
custom_aggregation = {}
custom_aggregation["Rank"] = lambda x: len(x)
data2 = data1.groupby("Year").agg(custom_aggregation)
data2.columns = ["Game Count"]
data2['Year'] = data2.index

labels = data2['Year'].tolist()
values = data2['Game Count'].tolist()

fig = px.pie(data2, values=values, names=labels)
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig['layout'].update(height=500, width=700, title='Game Count by Year', legend_title="Year")
fig.show()

- 2009 was the year which have most game count, followed by 2008 and 2010

# Platform & Genre Sales Contribution in Each Region

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, 
                    cols=2, 
                    specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=('Platfrom',
                                    'Genre'))


custom_aggregation = {}
custom_aggregation["Global_Sales"] = "mean"
data2 = data1.groupby("Platform").agg(custom_aggregation)
data2.columns = ["Global Sales"]
data2["Global Sales"] = data2["Global Sales"]*1000 
data2['Platform'] = data2.index

labels = data2['Platform'].tolist()
values = data2['Global Sales'].tolist()

fig.add_trace(go.Pie(
                    labels=labels,
                    values=values, 
                    name=''),
                    1,1)

#-------------------------------------------------------------

custom_aggregation = {}
custom_aggregation["Global_Sales"] = "mean"
data2 = data1.groupby("Genre").agg(custom_aggregation)
data2.columns = ["Global Sales"]
data2["Global Sales"] = data2["Global Sales"]*1000 
data2['Genre'] = data2.index

labels = data2['Genre'].tolist()
values = data2['Global Sales'].tolist()

fig.add_trace(go.Pie(
                    labels=labels,
                    values=values, 
                    name= ''),
                    1,2)

fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig['layout'].update(height=500, 
                     width=800, 
                     title='Global Sales Contribution by:',
                    legend_title="Platfrom")
fig.show()



In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, 
                    cols=2, 
                    specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=('Platfrom',
                                    'Genre'))


custom_aggregation = {}
custom_aggregation["NA_Sales"] = "mean"
data2 = data1.groupby("Platform").agg(custom_aggregation)
data2.columns = ["North America Sales"]
data2["North America Sales"] = data2["North America Sales"]*1000 
data2['Platform'] = data2.index

labels = data2['Platform'].tolist()
values = data2['North America Sales'].tolist()

fig.add_trace(go.Pie(
                    labels=labels,
                    values=values, 
                    name=''),
                    1,1)

#-------------------------------------------------------------

custom_aggregation = {}
custom_aggregation["NA_Sales"] = "mean"
data2 = data1.groupby("Genre").agg(custom_aggregation)
data2.columns = ["North America Sales"]
data2["North America Sales"] = data2["North America Sales"]*1000 
data2['Genre'] = data2.index

labels = data2['Genre'].tolist()
values = data2['North America Sales'].tolist()

fig.add_trace(go.Pie(
                    labels=labels,
                    values=values, 
                    name= ''),
                    1,2)

fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig['layout'].update(height=500, 
                     width=800, 
                     title='North America Sales Contribution by:',
                    legend_title="Platfrom")
fig.show()

In [ ]:
fig = make_subplots(rows=1, 
                    cols=2, 
                    specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=('Platfrom',
                                    'Genre'))


custom_aggregation = {}
custom_aggregation["EU_Sales"] = "mean"
data2 = data1.groupby("Platform").agg(custom_aggregation)
data2.columns = ["Europe Sales"]
data2["Europe Sales"] = data2["Europe Sales"]*1000 
data2['Platform'] = data2.index

labels = data2['Platform'].tolist()
values = data2['Europe Sales'].tolist()

fig.add_trace(go.Pie(
                    labels=labels,
                    values=values, 
                    name=''),
                    1,1)

#-------------------------------------------------------------

custom_aggregation = {}
custom_aggregation["EU_Sales"] = "mean"
data2 = data1.groupby("Genre").agg(custom_aggregation)
data2.columns = ["Europe Sales"]
data2["Europe Sales"] = data2["Europe Sales"]*1000 
data2['Genre'] = data2.index

labels = data2['Genre'].tolist()
values = data2['Europe Sales'].tolist()

fig.add_trace(go.Pie(
                    labels=labels,
                    values=values, 
                    name= ''),
                    1,2)

fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig['layout'].update(height=500, 
                     width=800, 
                     title='Europe Sales Contribution by:',
                    legend_title="Platfrom")
fig.show()

In [ ]:
fig = make_subplots(rows=1, 
                    cols=2, 
                    specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=('Platfrom',
                                    'Genre'))


custom_aggregation = {}
custom_aggregation["JP_Sales"] = "mean"
data2 = data1.groupby("Platform").agg(custom_aggregation)
data2.columns = ["Japan Sales"]
data2["Japan Sales"] = data2["Japan Sales"]*1000 
data2['Platform'] = data2.index

labels = data2['Platform'].tolist()
values = data2['Japan Sales'].tolist()

fig.add_trace(go.Pie(
                    labels=labels,
                    values=values, 
                    name=''),
                    1,1)

#-------------------------------------------------------------

custom_aggregation = {}
custom_aggregation["JP_Sales"] = "mean"
data2 = data1.groupby("Genre").agg(custom_aggregation)
data2.columns = ["Japan Sales"]
data2["Japan Sales"] = data2["Japan Sales"]*1000 
data2['Genre'] = data2.index

labels = data2['Genre'].tolist()
values = data2['Japan Sales'].tolist()

fig.add_trace(go.Pie(
                    labels=labels,
                    values=values, 
                    name= ''),
                    1,2)

fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig['layout'].update(height=500, 
                     width=800, 
                     title='Japan Sales Contribution by:',
                    legend_title="Platfrom")
fig.show()

In [ ]:
fig = make_subplots(rows=1, 
                    cols=2, 
                    specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=('Platfrom',
                                    'Genre'))


custom_aggregation = {}
custom_aggregation["Other_Sales"] = "mean"
data2 = data1.groupby("Platform").agg(custom_aggregation)
data2.columns = ["Other Sales"]
data2["Other Sales"] = data2["Other Sales"]*1000 
data2['Platform'] = data2.index

labels = data2['Platform'].tolist()
values = data2['Other Sales'].tolist()

fig.add_trace(go.Pie(
                    labels=labels,
                    values=values, 
                    name=''),
                    1,1)

#-------------------------------------------------------------

custom_aggregation = {}
custom_aggregation["Other_Sales"] = "mean"
data2 = data1.groupby("Genre").agg(custom_aggregation)
data2.columns = ["Other Sales"]
data2["Other Sales"] = data2["Other Sales"]*1000 
data2['Genre'] = data2.index

labels = data2['Genre'].tolist()
values = data2['Other Sales'].tolist()

fig.add_trace(go.Pie(
                    labels=labels,
                    values=values, 
                    name= ''),
                    1,2)

fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig['layout'].update(height=500, 
                     width=800, 
                     title='Other Sales Contribution by:',
                    legend_title="Platfrom")
fig.show()

# Genre Countplot and Top Ranking

In [ ]:
custom_aggregation = {}
custom_aggregation["Rank"] = lambda x: len(x)
data0 = data1.groupby("Genre").agg(custom_aggregation)

custom_aggregation = {}
custom_aggregation["Rank"] = ['min', 'median', 'max']
data0_ = data1.groupby("Genre").agg(custom_aggregation)

data0 = pd.concat([data0,data0_], axis=1)
data0.columns = ["Count",'Min Rank','Median Rank','Max Rank']
data0['Genre'] = data0.index

fig = px.bar(data0, x='Genre', y="Count", color="Genre")
fig['layout'].update(height=400, width=700, title='Genre Countplot')
fig.show()

In [ ]:
data0 = data0.sort_values(by='Min Rank')
x = data0['Genre'].tolist()
y = data0['Min Rank'].tolist()

data = dict(
            number=y,
            stage=x)

fig = px.funnel(data, x='number', y='stage', color='number')
fig['layout'].update(height=500, 
                     width=600, 
                     title='Top Ranking Based on Genre', 
                    xaxis_title="Rank",
                    yaxis_title="Genre",
                    legend_title="Rank")
fig.show()

# Platform Countplot and Top Ranking

In [ ]:
custom_aggregation = {}
custom_aggregation["Rank"] = lambda x: len(x)
data0 = data1.groupby("Platform").agg(custom_aggregation)

custom_aggregation = {}
custom_aggregation["Rank"] = ['min', 'median', 'max']
data0_ = data1.groupby("Platform").agg(custom_aggregation)

data0 = pd.concat([data0,data0_], axis=1)
data0.columns = ["Count",'Min Rank','Median Rank','Max Rank']
data0['Platform'] = data0.index
data0 = data0.sort_values(by='Min Rank')

fig = px.bar(data0, x='Platform', y="Count", color="Platform")
fig['layout'].update(height=400, width=700, title='Platform Countplot')
fig.show()

In [ ]:
x = data0['Platform'].tolist()[:10]
y = data0['Min Rank'].tolist()[:10]

data = dict(
            number=y,
            stage=x)

fig = px.funnel(data, x='number', y='stage', color='number')
fig['layout'].update(height=500, 
                     width=600, 
                     title='Top Ranking Based on Platform', 
                    xaxis_title="Rank",
                    yaxis_title="Platform",
                    legend_title="Rank")
fig.show()

# Publisher Countplot and Top Ranking

In [ ]:
custom_aggregation = {}
custom_aggregation["Rank"] = lambda x: len(x)
data0 = data1.groupby("Publisher").agg(custom_aggregation)

custom_aggregation = {}
custom_aggregation["Rank"] = ['min', 'median', 'max']
data0_ = data1.groupby("Publisher").agg(custom_aggregation)

data0 = pd.concat([data0,data0_], axis=1)
data0.columns = ["Count",'Min Rank','Median Rank','Max Rank']
data0['Publisher'] = data0.index
data0 = data0.sort_values(by='Count', ascending=False)[:20]


fig = px.bar(data0, x='Publisher', y="Count", color="Publisher")
fig['layout'].update(height=700, width=900, title='Top 10 Publisher by Size')
fig.show()

In [ ]:
custom_aggregation = {}
custom_aggregation["Rank"] = lambda x: len(x)
data0 = data1.groupby("Publisher").agg(custom_aggregation)

custom_aggregation = {}
custom_aggregation["Rank"] = ['min', 'median', 'max']
data0_ = data1.groupby("Publisher").agg(custom_aggregation)

data0 = pd.concat([data0,data0_], axis=1)
data0.columns = ["Count",'Min Rank','Median Rank','Max Rank']
data0['Publisher'] = data0.index
data0 = data0.sort_values(by='Min Rank', ascending=True)[:20]

x = data0['Publisher'].tolist()[:10]
y = data0['Min Rank'].tolist()[:10]

data = dict(
            number=y,
            stage=x)

fig = px.funnel(data, x='number', y='stage', color='number')
fig['layout'].update(height=500, 
                     width=600, 
                     title='Top Ranking Based on Publisher', 
                    xaxis_title="Rank",
                    yaxis_title="Platform",
                    legend_title="Rank")
fig.show()

- We can see that Nintendo was in first place in top ranking, but 4th place by size. Can we assume that Nintendo offering Quality over Quantity?

# Year on Year Sales

In [ ]:
'''Equidistance Checking'''

data1 = data1.sort_values(by='Year')
data1['Time_Interval'] = data1.Year - data1.Year.shift(1)
data1[['Year', 'Time_Interval']].head()

print(f"{data1['Time_Interval'].value_counts()}")
data1 = data1.drop('Time_Interval', axis=1)

In [ ]:
data1['Year'].min(), data1['Year'].max()

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, 
                    cols=1)

custom_aggregation = {}
custom_aggregation["Global_Sales"] = "mean"
custom_aggregation["NA_Sales"] = "mean"
custom_aggregation["EU_Sales"] = "mean"
custom_aggregation["JP_Sales"] = "mean"
custom_aggregation["Other_Sales"] = "mean"
data2 = data1.groupby("Year").agg(custom_aggregation)

data2.columns = ["Global Sales", 'North America Sales','Europe Sales','Japan Sales','Other Sales']

data2["Global Sales"] = data2["Global Sales"]*1000000
data2["North America Sales"] = data2["North America Sales"]*1000000
data2["Europe Sales"] = data2["Europe Sales"]*1000000
data2["Japan Sales"] = data2["Japan Sales"]*1000000
data2["Other Sales"] = data2["Other Sales"]*1000000

data2['Year'] = data2.index


x = data2['Year'].tolist()
y = data2['Global Sales'].tolist()
y_na = data2['North America Sales'].tolist()
y_eu = data2['Europe Sales'].tolist()
y_jp = data2['Japan Sales'].tolist()
y_oth = data2['Other Sales'].tolist()


fig.add_trace(go.Scatter(x=x, y=y,name='Global'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_na,name='North America'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_eu,name='Europe Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_jp,name='Japan Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_oth,name='Other Sales'), 1, 1)

fig['layout'].update(height=500, 
                     width=900, 
                     title='Overall YoY Sales',
                    xaxis_title="Year",
                    yaxis_title="Sales",
                    legend_title="Region")
fig.show()

In [ ]:
# Genre: Shooter

fig = make_subplots(rows=1, 
                    cols=1)

data_ = data1.loc[data1['Genre'] == 'Shooter']

custom_aggregation = {}
custom_aggregation["Global_Sales"] = "mean"
custom_aggregation["NA_Sales"] = "mean"
custom_aggregation["EU_Sales"] = "mean"
custom_aggregation["JP_Sales"] = "mean"
custom_aggregation["Other_Sales"] = "mean"
data2 = data_.groupby("Year").agg(custom_aggregation)

data2.columns = ["Global Sales", 'North America Sales','Europe Sales','Japan Sales','Other Sales']

data2["Global Sales"] = data2["Global Sales"]*1000000
data2["North America Sales"] = data2["North America Sales"]*1000000
data2["Europe Sales"] = data2["Europe Sales"]*1000000
data2["Japan Sales"] = data2["Japan Sales"]*1000000
data2["Other Sales"] = data2["Other Sales"]*1000000

data2['Year'] = data2.index


x = data2['Year'].tolist()
y = data2['Global Sales'].tolist()
y_na = data2['North America Sales'].tolist()
y_eu = data2['Europe Sales'].tolist()
y_jp = data2['Japan Sales'].tolist()
y_oth = data2['Other Sales'].tolist()


fig.add_trace(go.Scatter(x=x, y=y,name='Global'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_na,name='North America'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_eu,name='Europe Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_jp,name='Japan Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_oth,name='Other Sales'), 1, 1)

fig['layout'].update(height=500, 
                     width=900, 
                     title='YoY Sales based on Genre: Shooter',
                    xaxis_title="Year",
                    yaxis_title="Sales",
                    legend_title="Region")
fig.show()

In [ ]:
# Genre: Misc

fig = make_subplots(rows=1, 
                    cols=1)

data_ = data1.loc[data1['Genre'] == 'Misc']

custom_aggregation = {}
custom_aggregation["Global_Sales"] = "mean"
custom_aggregation["NA_Sales"] = "mean"
custom_aggregation["EU_Sales"] = "mean"
custom_aggregation["JP_Sales"] = "mean"
custom_aggregation["Other_Sales"] = "mean"
data2 = data_.groupby("Year").agg(custom_aggregation)

data2.columns = ["Global Sales", 'North America Sales','Europe Sales','Japan Sales','Other Sales']

data2["Global Sales"] = data2["Global Sales"]*1000000
data2["North America Sales"] = data2["North America Sales"]*1000000
data2["Europe Sales"] = data2["Europe Sales"]*1000000
data2["Japan Sales"] = data2["Japan Sales"]*1000000
data2["Other Sales"] = data2["Other Sales"]*1000000

data2['Year'] = data2.index


x = data2['Year'].tolist()
y = data2['Global Sales'].tolist()
y_na = data2['North America Sales'].tolist()
y_eu = data2['Europe Sales'].tolist()
y_jp = data2['Japan Sales'].tolist()
y_oth = data2['Other Sales'].tolist()


fig.add_trace(go.Scatter(x=x, y=y,name='Global'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_na,name='North America'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_eu,name='Europe Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_jp,name='Japan Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_oth,name='Other Sales'), 1, 1)

fig['layout'].update(height=500, 
                     width=900, 
                     title='YoY Sales based on Genre: Misc',
                    xaxis_title="Year",
                    yaxis_title="Sales",
                    legend_title="Region")
fig.show()

In [ ]:
# Genre: Action

fig = make_subplots(rows=1, 
                    cols=1)

data_ = data1.loc[data1['Genre'] == 'Action']

custom_aggregation = {}
custom_aggregation["Global_Sales"] = "mean"
custom_aggregation["NA_Sales"] = "mean"
custom_aggregation["EU_Sales"] = "mean"
custom_aggregation["JP_Sales"] = "mean"
custom_aggregation["Other_Sales"] = "mean"
data2 = data_.groupby("Year").agg(custom_aggregation)

data2.columns = ["Global Sales", 'North America Sales','Europe Sales','Japan Sales','Other Sales']

data2["Global Sales"] = data2["Global Sales"]*1000000
data2["North America Sales"] = data2["North America Sales"]*1000000
data2["Europe Sales"] = data2["Europe Sales"]*1000000
data2["Japan Sales"] = data2["Japan Sales"]*1000000
data2["Other Sales"] = data2["Other Sales"]*1000000

data2['Year'] = data2.index


x = data2['Year'].tolist()
y = data2['Global Sales'].tolist()
y_na = data2['North America Sales'].tolist()
y_eu = data2['Europe Sales'].tolist()
y_jp = data2['Japan Sales'].tolist()
y_oth = data2['Other Sales'].tolist()


fig.add_trace(go.Scatter(x=x, y=y,name='Global'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_na,name='North America'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_eu,name='Europe Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_jp,name='Japan Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_oth,name='Other Sales'), 1, 1)

fig['layout'].update(height=500, 
                     width=900, 
                     title='YoY Sales based on Genre: Action',
                    xaxis_title="Year",
                    yaxis_title="Sales",
                    legend_title="Region")
fig.show()

In [ ]:
# Genre: Sports

fig = make_subplots(rows=1, 
                    cols=1)

data_ = data1.loc[data1['Genre'] == 'Sports']

custom_aggregation = {}
custom_aggregation["Global_Sales"] = "mean"
custom_aggregation["NA_Sales"] = "mean"
custom_aggregation["EU_Sales"] = "mean"
custom_aggregation["JP_Sales"] = "mean"
custom_aggregation["Other_Sales"] = "mean"
data2 = data_.groupby("Year").agg(custom_aggregation)

data2.columns = ["Global Sales", 'North America Sales','Europe Sales','Japan Sales','Other Sales']

data2["Global Sales"] = data2["Global Sales"]*1000000
data2["North America Sales"] = data2["North America Sales"]*1000000
data2["Europe Sales"] = data2["Europe Sales"]*1000000
data2["Japan Sales"] = data2["Japan Sales"]*1000000
data2["Other Sales"] = data2["Other Sales"]*1000000

data2['Year'] = data2.index


x = data2['Year'].tolist()
y = data2['Global Sales'].tolist()
y_na = data2['North America Sales'].tolist()
y_eu = data2['Europe Sales'].tolist()
y_jp = data2['Japan Sales'].tolist()
y_oth = data2['Other Sales'].tolist()


fig.add_trace(go.Scatter(x=x, y=y,name='Global'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_na,name='North America'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_eu,name='Europe Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_jp,name='Japan Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_oth,name='Other Sales'), 1, 1)

fig['layout'].update(height=500, 
                     width=900, 
                     title='YoY Sales based on Genre: Sports',
                    xaxis_title="Year",
                    yaxis_title="Sales",
                    legend_title="Region")
fig.show()

In [ ]:
# Genre: Fighting

fig = make_subplots(rows=1, 
                    cols=1)

data_ = data1.loc[data1['Genre'] == 'Fighting']

custom_aggregation = {}
custom_aggregation["Global_Sales"] = "mean"
custom_aggregation["NA_Sales"] = "mean"
custom_aggregation["EU_Sales"] = "mean"
custom_aggregation["JP_Sales"] = "mean"
custom_aggregation["Other_Sales"] = "mean"
data2 = data_.groupby("Year").agg(custom_aggregation)

data2.columns = ["Global Sales", 'North America Sales','Europe Sales','Japan Sales','Other Sales']

data2["Global Sales"] = data2["Global Sales"]*1000000
data2["North America Sales"] = data2["North America Sales"]*1000000
data2["Europe Sales"] = data2["Europe Sales"]*1000000
data2["Japan Sales"] = data2["Japan Sales"]*1000000
data2["Other Sales"] = data2["Other Sales"]*1000000

data2['Year'] = data2.index


x = data2['Year'].tolist()
y = data2['Global Sales'].tolist()
y_na = data2['North America Sales'].tolist()
y_eu = data2['Europe Sales'].tolist()
y_jp = data2['Japan Sales'].tolist()
y_oth = data2['Other Sales'].tolist()


fig.add_trace(go.Scatter(x=x, y=y,name='Global'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_na,name='North America'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_eu,name='Europe Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_jp,name='Japan Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_oth,name='Other Sales'), 1, 1)

fig['layout'].update(height=500, 
                     width=900, 
                     title='YoY Sales based on Genre: Fighting',
                    xaxis_title="Year",
                    yaxis_title="Sales",
                    legend_title="Region")
fig.show()

In [ ]:
# Genre: Racing

fig = make_subplots(rows=1, 
                    cols=1)

data_ = data1.loc[data1['Genre'] == 'Racing']

custom_aggregation = {}
custom_aggregation["Global_Sales"] = "mean"
custom_aggregation["NA_Sales"] = "mean"
custom_aggregation["EU_Sales"] = "mean"
custom_aggregation["JP_Sales"] = "mean"
custom_aggregation["Other_Sales"] = "mean"
data2 = data_.groupby("Year").agg(custom_aggregation)

data2.columns = ["Global Sales", 'North America Sales','Europe Sales','Japan Sales','Other Sales']

data2["Global Sales"] = data2["Global Sales"]*1000000
data2["North America Sales"] = data2["North America Sales"]*1000000
data2["Europe Sales"] = data2["Europe Sales"]*1000000
data2["Japan Sales"] = data2["Japan Sales"]*1000000
data2["Other Sales"] = data2["Other Sales"]*1000000

data2['Year'] = data2.index


x = data2['Year'].tolist()
y = data2['Global Sales'].tolist()
y_na = data2['North America Sales'].tolist()
y_eu = data2['Europe Sales'].tolist()
y_jp = data2['Japan Sales'].tolist()
y_oth = data2['Other Sales'].tolist()


fig.add_trace(go.Scatter(x=x, y=y,name='Global'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_na,name='North America'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_eu,name='Europe Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_jp,name='Japan Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_oth,name='Other Sales'), 1, 1)

fig['layout'].update(height=500, 
                     width=900, 
                     title='YoY Sales based on Genre: Racing',
                    xaxis_title="Year",
                    yaxis_title="Sales",
                    legend_title="Region")
fig.show()

In [ ]:
# Genre: Platform

fig = make_subplots(rows=1, 
                    cols=1)

data_ = data1.loc[data1['Genre'] == 'Platform']

custom_aggregation = {}
custom_aggregation["Global_Sales"] = "mean"
custom_aggregation["NA_Sales"] = "mean"
custom_aggregation["EU_Sales"] = "mean"
custom_aggregation["JP_Sales"] = "mean"
custom_aggregation["Other_Sales"] = "mean"
data2 = data_.groupby("Year").agg(custom_aggregation)

data2.columns = ["Global Sales", 'North America Sales','Europe Sales','Japan Sales','Other Sales']

data2["Global Sales"] = data2["Global Sales"]*1000000
data2["North America Sales"] = data2["North America Sales"]*1000000
data2["Europe Sales"] = data2["Europe Sales"]*1000000
data2["Japan Sales"] = data2["Japan Sales"]*1000000
data2["Other Sales"] = data2["Other Sales"]*1000000

data2['Year'] = data2.index


x = data2['Year'].tolist()
y = data2['Global Sales'].tolist()
y_na = data2['North America Sales'].tolist()
y_eu = data2['Europe Sales'].tolist()
y_jp = data2['Japan Sales'].tolist()
y_oth = data2['Other Sales'].tolist()


fig.add_trace(go.Scatter(x=x, y=y,name='Global'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_na,name='North America'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_eu,name='Europe Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_jp,name='Japan Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_oth,name='Other Sales'), 1, 1)

fig['layout'].update(height=500, 
                     width=900, 
                     title='YoY Sales based on Genre: Platform',
                    xaxis_title="Year",
                    yaxis_title="Sales",
                    legend_title="Region")
fig.show()

In [ ]:
# Genre: Puzzle

fig = make_subplots(rows=1, 
                    cols=1)

data_ = data1.loc[data1['Genre'] == 'Puzzle']

custom_aggregation = {}
custom_aggregation["Global_Sales"] = "mean"
custom_aggregation["NA_Sales"] = "mean"
custom_aggregation["EU_Sales"] = "mean"
custom_aggregation["JP_Sales"] = "mean"
custom_aggregation["Other_Sales"] = "mean"
data2 = data_.groupby("Year").agg(custom_aggregation)

data2.columns = ["Global Sales", 'North America Sales','Europe Sales','Japan Sales','Other Sales']

data2["Global Sales"] = data2["Global Sales"]*1000000
data2["North America Sales"] = data2["North America Sales"]*1000000
data2["Europe Sales"] = data2["Europe Sales"]*1000000
data2["Japan Sales"] = data2["Japan Sales"]*1000000
data2["Other Sales"] = data2["Other Sales"]*1000000

data2['Year'] = data2.index


x = data2['Year'].tolist()
y = data2['Global Sales'].tolist()
y_na = data2['North America Sales'].tolist()
y_eu = data2['Europe Sales'].tolist()
y_jp = data2['Japan Sales'].tolist()
y_oth = data2['Other Sales'].tolist()


fig.add_trace(go.Scatter(x=x, y=y,name='Global'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_na,name='North America'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_eu,name='Europe Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_jp,name='Japan Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_oth,name='Other Sales'), 1, 1)

fig['layout'].update(height=500, 
                     width=900, 
                     title='YoY Sales based on Genre: Puzzle',
                    xaxis_title="Year",
                    yaxis_title="Sales",
                    legend_title="Region")
fig.show()

In [ ]:
# Genre: Simulation

fig = make_subplots(rows=1, 
                    cols=1)

data_ = data1.loc[data1['Genre'] == 'Simulation']

custom_aggregation = {}
custom_aggregation["Global_Sales"] = "mean"
custom_aggregation["NA_Sales"] = "mean"
custom_aggregation["EU_Sales"] = "mean"
custom_aggregation["JP_Sales"] = "mean"
custom_aggregation["Other_Sales"] = "mean"
data2 = data_.groupby("Year").agg(custom_aggregation)

data2.columns = ["Global Sales", 'North America Sales','Europe Sales','Japan Sales','Other Sales']

data2["Global Sales"] = data2["Global Sales"]*1000000
data2["North America Sales"] = data2["North America Sales"]*1000000
data2["Europe Sales"] = data2["Europe Sales"]*1000000
data2["Japan Sales"] = data2["Japan Sales"]*1000000
data2["Other Sales"] = data2["Other Sales"]*1000000

data2['Year'] = data2.index


x = data2['Year'].tolist()
y = data2['Global Sales'].tolist()
y_na = data2['North America Sales'].tolist()
y_eu = data2['Europe Sales'].tolist()
y_jp = data2['Japan Sales'].tolist()
y_oth = data2['Other Sales'].tolist()


fig.add_trace(go.Scatter(x=x, y=y,name='Global'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_na,name='North America'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_eu,name='Europe Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_jp,name='Japan Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_oth,name='Other Sales'), 1, 1)

fig['layout'].update(height=500, 
                     width=900, 
                     title='YoY Sales based on Genre: Simulation',
                    xaxis_title="Year",
                    yaxis_title="Sales",
                    legend_title="Region")
fig.show()

In [ ]:
# Genre: Adventure

fig = make_subplots(rows=1, 
                    cols=1)

data_ = data1.loc[data1['Genre'] == 'Adventure']

custom_aggregation = {}
custom_aggregation["Global_Sales"] = "mean"
custom_aggregation["NA_Sales"] = "mean"
custom_aggregation["EU_Sales"] = "mean"
custom_aggregation["JP_Sales"] = "mean"
custom_aggregation["Other_Sales"] = "mean"
data2 = data_.groupby("Year").agg(custom_aggregation)

data2.columns = ["Global Sales", 'North America Sales','Europe Sales','Japan Sales','Other Sales']

data2["Global Sales"] = data2["Global Sales"]*1000000
data2["North America Sales"] = data2["North America Sales"]*1000000
data2["Europe Sales"] = data2["Europe Sales"]*1000000
data2["Japan Sales"] = data2["Japan Sales"]*1000000
data2["Other Sales"] = data2["Other Sales"]*1000000

data2['Year'] = data2.index


x = data2['Year'].tolist()
y = data2['Global Sales'].tolist()
y_na = data2['North America Sales'].tolist()
y_eu = data2['Europe Sales'].tolist()
y_jp = data2['Japan Sales'].tolist()
y_oth = data2['Other Sales'].tolist()


fig.add_trace(go.Scatter(x=x, y=y,name='Global'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_na,name='North America'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_eu,name='Europe Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_jp,name='Japan Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_oth,name='Other Sales'), 1, 1)

fig['layout'].update(height=500, 
                     width=900, 
                     title='YoY Sales based on Genre: Adventure',
                    xaxis_title="Year",
                    yaxis_title="Sales",
                    legend_title="Region")
fig.show()

In [ ]:
# Genre: Role-Playing

fig = make_subplots(rows=1, 
                    cols=1)

data_ = data1.loc[data1['Genre'] == 'Role-Playing']

custom_aggregation = {}
custom_aggregation["Global_Sales"] = "mean"
custom_aggregation["NA_Sales"] = "mean"
custom_aggregation["EU_Sales"] = "mean"
custom_aggregation["JP_Sales"] = "mean"
custom_aggregation["Other_Sales"] = "mean"
data2 = data_.groupby("Year").agg(custom_aggregation)

data2.columns = ["Global Sales", 'North America Sales','Europe Sales','Japan Sales','Other Sales']

data2["Global Sales"] = data2["Global Sales"]*1000000
data2["North America Sales"] = data2["North America Sales"]*1000000
data2["Europe Sales"] = data2["Europe Sales"]*1000000
data2["Japan Sales"] = data2["Japan Sales"]*1000000
data2["Other Sales"] = data2["Other Sales"]*1000000

data2['Year'] = data2.index


x = data2['Year'].tolist()
y = data2['Global Sales'].tolist()
y_na = data2['North America Sales'].tolist()
y_eu = data2['Europe Sales'].tolist()
y_jp = data2['Japan Sales'].tolist()
y_oth = data2['Other Sales'].tolist()


fig.add_trace(go.Scatter(x=x, y=y,name='Global'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_na,name='North America'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_eu,name='Europe Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_jp,name='Japan Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_oth,name='Other Sales'), 1, 1)

fig['layout'].update(height=500, 
                     width=900, 
                     title='YoY Sales based on Genre: Role-Playing',
                    xaxis_title="Year",
                    yaxis_title="Sales",
                    legend_title="Region")
fig.show()

In [ ]:
# Genre: Strategy

fig = make_subplots(rows=1, 
                    cols=1)

data_ = data1.loc[data1['Genre'] == 'Strategy']

custom_aggregation = {}
custom_aggregation["Global_Sales"] = "mean"
custom_aggregation["NA_Sales"] = "mean"
custom_aggregation["EU_Sales"] = "mean"
custom_aggregation["JP_Sales"] = "mean"
custom_aggregation["Other_Sales"] = "mean"
data2 = data_.groupby("Year").agg(custom_aggregation)

data2.columns = ["Global Sales", 'North America Sales','Europe Sales','Japan Sales','Other Sales']

data2["Global Sales"] = data2["Global Sales"]*1000000
data2["North America Sales"] = data2["North America Sales"]*1000000
data2["Europe Sales"] = data2["Europe Sales"]*1000000
data2["Japan Sales"] = data2["Japan Sales"]*1000000
data2["Other Sales"] = data2["Other Sales"]*1000000

data2['Year'] = data2.index


x = data2['Year'].tolist()
y = data2['Global Sales'].tolist()
y_na = data2['North America Sales'].tolist()
y_eu = data2['Europe Sales'].tolist()
y_jp = data2['Japan Sales'].tolist()
y_oth = data2['Other Sales'].tolist()


fig.add_trace(go.Scatter(x=x, y=y,name='Global'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_na,name='North America'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_eu,name='Europe Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_jp,name='Japan Sales'), 1, 1)
fig.add_trace(go.Scatter(x=x, y=y_oth,name='Other Sales'), 1, 1)

fig['layout'].update(height=500, 
                     width=900, 
                     title='YoY Sales based on Genre: Strategy',
                    xaxis_title="Year",
                    yaxis_title="Sales",
                    legend_title="Region")
fig.show()

- We can see that every game gender have specific peak, in it was interseting to find out whic game was populer in that particular year!

In [ ]:
# Shooter on 1984
import seaborn as sns

cm = sns.light_palette("green", as_cmap=True)
data_1 = data1[(data1['Year'] == 1984) & (data1['Genre'] == 'Shooter')].sort_values(by='Global_Sales', ascending=False)
data_1.style.background_gradient(cmap = cm)

Duck Hunt Preview:
- Duck Hunt is responsible for a lot of the early success of the NES, but then Mario took over and they ran with that. It’s interesting to think how, despite its success, Duck Hunt never got a sequel.
- It seems weird they would never put out another version since there were limited options for games you could use with the Zapper.
- All in all, Duck Hunt remains one of the best-selling video games of all time, coming in at number three with 28.31 million units sold. And this is just one game that was released in 1984.

(https://medium.com/everything-80s/the-story-of-duck-hunt-the-original-backbone-of-the-nes-6d4b18c0ff89)

In [ ]:
# Misc on 1992

cm = sns.light_palette("green", as_cmap=True)
data_2 = data1[(data1['Year'] == 1992) & (data1['Genre'] == 'Misc')].sort_values(by='Global_Sales', ascending=False)
data_2.style.background_gradient(cmap = cm)

In [ ]:
# Action on 1992

cm = sns.light_palette("green", as_cmap=True)
data_3 = data1[(data1['Year'] == 1992) & (data1['Genre'] == 'Action')].sort_values(by='Global_Sales', ascending=False)
data_3.style.background_gradient(cmap = cm)

In [ ]:
# Sports on 1983

cm = sns.light_palette("green", as_cmap=True)
data_4 = data1[(data1['Year'] == 1983) & (data1['Genre'] == 'Sports')].sort_values(by='Global_Sales', ascending=False)
data_4.style.background_gradient(cmap = cm)

In [ ]:
# Fighting on 1987

cm = sns.light_palette("green", as_cmap=True)
data_5 = data1[(data1['Year'] == 1987) & (data1['Genre'] == 'Fighting')].sort_values(by='Global_Sales', ascending=False)
data_5.style.background_gradient(cmap = cm)

In [ ]:
# Racing on 1992

cm = sns.light_palette("green", as_cmap=True)
data_6 = data1[(data1['Year'] == 1992) & (data1['Genre'] == 'Racing')].sort_values(by='Global_Sales', ascending=False)
data_6.style.background_gradient(cmap = cm)

In [ ]:
# Platform on 1992

cm = sns.light_palette("green", as_cmap=True)
data_ = data1[(data1['Year'] == 1992) & (data1['Genre'] == 'Platform')].sort_values(by='Global_Sales', ascending=False)[:3]
data_.style.background_gradient(cmap = cm)

In [ ]:
# Adventure on 1987

cm = sns.light_palette("green", as_cmap=True)
data_ = data1[(data1['Year'] == 1987) & (data1['Genre'] == 'Adventure')].sort_values(by='Global_Sales', ascending=False)
data_.style.background_gradient(cmap = cm)

In [ ]:
# Role-Playing on 1990

cm = sns.light_palette("green", as_cmap=True)
data_ = data1[(data1['Year'] == 1990) & (data1['Genre'] == 'Role-Playing')].sort_values(by='Global_Sales', ascending=False)
data_.style.background_gradient(cmap = cm)

In [ ]:
# Strategy on 1999

cm = sns.light_palette("green", as_cmap=True)
data_ = data1[(data1['Year'] == 1999) & (data1['Genre'] == 'Strategy')].sort_values(by='Global_Sales', ascending=False)[:3]
data_.style.background_gradient(cmap = cm)

# Who's All Time Best?

In [ ]:
data_rank = data1.sort_values(by='Global_Sales', ascending=False)[:100]
data_rank.head()

In [ ]:
fig = make_subplots(rows=1, 
                    cols=2, 
                    specs=[[{'type':'pie','colspan':2},None]])

# specs=[[{"type": "pie"},{"type": "pie"},{"type": "pie"}],
#        [{"type": "pie"},{"type": "pie"},{"type": "pie"}],
#        [{"type": "pie"},{"type": "bar",'colspan':2},None]]


custom_aggregation = {}
custom_aggregation["Global_Sales"] = "mean"
data2 = data_rank.groupby("Publisher").agg(custom_aggregation)
data2.columns = ["Global Sales"]
data2["Global Sales"] = data2["Global Sales"]*1000000
data2['Publisher'] = data2.index

labels = data2['Publisher'].tolist()
values = data2['Global Sales'].tolist()

fig.add_trace(go.Pie(
                    labels=labels,
                    values=values, 
                    name=''),
                    1,1)

fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig['layout'].update(height=500, 
                     width=800, 
                     title='All Time Top 100 Publisher',
                    legend_title="Publisher")
fig.show()

In [ ]:
fig = make_subplots(rows=1, 
                    cols=2, 
                    specs=[[{'type':'pie','colspan':2},None]])

custom_aggregation = {}
custom_aggregation["Global_Sales"] = "mean"
data2 = data_rank.groupby("Genre").agg(custom_aggregation)
data2.columns = ["Global Sales"]
data2["Global Sales"] = data2["Global Sales"]*1000000
data2['Genre'] = data2.index

labels = data2['Genre'].tolist()
values = data2['Global Sales'].tolist()

fig.add_trace(go.Pie(
                    labels=labels,
                    values=values, 
                    name=''),
                    1,1)

fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig['layout'].update(height=500, 
                     width=800, 
                     title='All Time Top 100 Genre',
                    legend_title="Genre")
fig.show()

In [ ]:
fig = make_subplots(rows=1, 
                    cols=2, 
                    specs=[[{'type':'pie','colspan':2},None]])

custom_aggregation = {}
custom_aggregation["Global_Sales"] = "mean"
data2 = data_rank.groupby("Platform").agg(custom_aggregation)
data2.columns = ["Global Sales"]
data2["Global Sales"] = data2["Global Sales"]*1000000
data2['Platform'] = data2.index

labels = data2['Platform'].tolist()
values = data2['Global Sales'].tolist()

fig.add_trace(go.Pie(
                    labels=labels,
                    values=values, 
                    name=''),
                    1,1)

fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig['layout'].update(height=500, 
                     width=800, 
                     title='All Time Top 100 Platform',
                    legend_title="Platform")
fig.show()

In [ ]:
data_rank = data_rank.sort_values(by='Global_Sales', ascending=False)[:10]

fig = px.bar(data_rank, x='Name', y="Global_Sales", color="Name")
fig['layout'].update(height=500, 
                     width=800, 
                     title='All Time Top 10 Games',
                     xaxis_title="Game",
                    yaxis_title="Sales",
                    legend_title="Game")
fig.show()

Conclusion:
- Every year have its 'booming' genre, which we already find out
- For all time best Publisher are:

    1. Nintendo: 15,2%
    2. Take-Two Interactive: 12,3%
    3. Microsoft Game Studio: 10,2%
    
    
- For all time best Genre are:

    1. Sports: 17,8%
    2. Puzzle: 10,8%
    3. Platform: 9,53%
    
    
- For all time best Platform are:

    1. NES: 10,7%
    2. Wii: 10,1%
    3. GB: 9,84%
    

- For all time best Games are:

    1. Wii Sports: 82,74 Million in Sales
    2. Super Mario Bros.: 40,24 Million in Sales
    3. Mario Kart Wii: 35,82 Million in Sales

- For answering the qustion, Does Nintendo is All Time Best VG Publisher? Yes, it does

Finish, dont forget to upvote! Thank you:)